# Import packages

We import the `pandas` package here too. In addition, we import `xml.etree.ElementTree`, which offers a XML parser that will prove very useful in our context!

In [1]:
# import packages
import pandas as pd
import xml.etree.ElementTree as et

# Set some jupyter display options
from IPython.display import display
pd.options.display.max_columns = None

# Get to know the data

Just like before, it is important to catch a glimpse of our source files to understand how they are built. Now we have  XML files that look like the following:

```xml
<?xml version="1.0" encoding="ISO-8859-1" standalone="yes"?>
<pdv_liste>
    <pdv id="1000001" latitude="4620114" longitude="519791" cp="01000" pop="R">
      <adresse>596 AVENUE DE TREVOUX</adresse>
      <ville>SAINT-DENIS-L&#xE8;S-BOURG</ville>
      <horaires automate-24-24="">
        <jour id="1" nom="Lundi" ferme=""/>
        <jour id="2" nom="Mardi" ferme=""/>
        <jour id="3" nom="Mercredi" ferme=""/>
        <jour id="4" nom="Jeudi" ferme=""/>
        <jour id="5" nom="Vendredi" ferme=""/>
        <jour id="6" nom="Samedi" ferme=""/>
        <jour id="7" nom="Dimanche" ferme=""/>
      </horaires>
      <services>
        <service>Station de gonflage</service>
        <service>Vente de gaz domestique (Butane, Propane)</service>
        <service>Automate CB</service>
      </services>
      <prix nom="Gazole" id="1" maj="2018-01-06T12:20:27" valeur="1376"/>
      <prix nom="Gazole" id="1" maj="2018-01-11T09:09:34" valeur="1389"/>
      <!-- ... (other "prix") -->
      <prix nom="SP98" id="6" maj="2018-12-27T09:53:35" valeur="1435"/>
      <prix nom="SP98" id="6" maj="2018-12-31T09:40:57" valeur="1435"/>
      <rupture id="6" nom="SP98" debut="2018-12-06T11:24:25" fin="2018-12-07T15:16:53"/>
      <rupture id="4" nom="GPLc" debut="2017-09-16T09:50:23" fin=""/>
      <rupture id="5" nom="E10" debut="2015-02-09T16:19:00" fin="2018-11-19T17:01:01"/>
      <!-- ... (other "rupture") -->
      <rupture id="5" nom="E10" debut="2018-12-13T09:49:49" fin=""/>
      <fermeture/>
    </pdv>
    <!-- ... (other "pdv") -->
</pdv_liste>

```

Lots of stuff indeed. We will use only a handful of the information therein. More precisely, we will use the following:

- `pdv`: a gas station = point of sale (*point de vente*)
  - `id`: the unique identified of the station
  - `latitude`, `longitude`: coordinates, multiplied by 100,000
  - `cp`: zip code (*code postal*)
  - `pop`: where the station is situated: `A` for a highway (*autoroute*), `R` for a road (*route*)
- `prix`: a price for a type of gas and a point in time
  - `nom`: type of gas
  - `maj`: last update date of the price (*there should be a timezone, let's assume it's in local time*)
  - `valeur`: price in euros, multiplied by 1,000

# Load and format the data

`xml.etree.ElementTree` works as follows: once the XML file is loaded in memory, take the **root** of the tree. Find all **child nodes** matching a certain criteria, and do something with them, like reading their **attributes** -- or iterating on their own children, recursively, until we have gathered all the information we wanted.

In a real case we would have several input files, namely `PrixCarburants_annuel_<YYYY>.xml`, `<YYYY>` being the year. They are structured in the same way. The best way to deal with this is to create a **function** that will read a file and return a data frame with all the interesting data of the corresponding year. Then we can call the function as many times as we want, easily.

In [2]:
# Prepare the list of attributes we will keep
PDV_ATTRIBUTES = ['''###CODE HERE###''']
PRIX_ATTRIBUTES = ['''###CODE HERE###''']

def read_prices_from_xml(input_path):
    # Load the XML file
    tree = et.parse(input_path)
    
    # Get a hand on the root
    pdv_liste = tree.getroot()
    
    # Each row will be a Python dictionary with {attribute: value}
    rows = []
    
    # Iterate over *stations*
    for pdv in pdv_liste.findall('''###CODE HERE###'''):      
        # Put the interesting attributes into a dictionary {pdv attribute: value}
        pdv_attrib = {key: pdv.attrib.get(key) for key in PDV_ATTRIBUTES}
        
        # Iterate over *prices* in pdv
        for prix in pdv.findall('''###CODE HERE###'''):
            # Put the interesting attributes into a dictionary {prix attribute: value}
            prix_attrib = {key: prix.attrib.get(key) for key in PRIX_ATTRIBUTES}
            
            # Merge the 2 small dictionaries into one
            # A new dictionary is created every time to prevent attributes from leaking between rows
            row = dict(pdv_attrib)
            row.update(prix_attrib)
            
            # Add the row to the list
            rows.append(row)
            
    return rows
    

def prices_to_data_frame(input_path):
    # Get the list of rows
    rows = read_prices_from_xml(input_path)
            
    # Create a data frame. Pandas is smart enough to see it gets dictionaries, and turns the keys into column names
    prices = pd.DataFrame(rows)

    # Finally return the result
    return prices

In [3]:
# A very small sample file is provided so you can test your code quickly
# The 2 "None" on rows with index 4 and 8 (price and gas_type, respectively) are expected
prices_to_data_frame('data/sample.xml')

,cp,id,latitude,longitude,maj,nom,pop,valeur
0,01000,1000001,4620114,519791,2018-11-05T10:54:17,SP95,R,1471
1,01000,1000001,4620114,519791,2018-11-05T10:54:18,SP95,R,1471
2,01000,1000001,4620114,519791,2018-11-10T10:43:04,SP95,R,1441
3,01000,1000002,4621842,522767,2018-01-03T09:55:42,Gazole,R,1374
4,01000,1000002,4621842,522767,2018-08-28T10:05:42,SP98,R,None
5,01000,1000002,4621842,522767,2018-12-31T06:21:48,SP98,R,1399
6,01000,1000009,4619566,522935,2018-01-02T08:29:19,Gazole,R,1367
7,01000,1000009,4619566,522935,2018-10-05T15:12:56,E10,R,1510
8,01000,1000009,4619566,522935,2018-10-06T09:57:08,None,R,1515
9,01000,1000009,4619566,522935,2018-10-13T09:48:44,E10,R,1509


# Save the results to a single CSV file

Now we know how to turn one XML file into a data frame. We will convert the 2019 prices and save the result.

Note the use of `%%time` at the beginning of the following cells. They are called *Jupyter (or IPython) magics* as they are not part of the Python language. They instruct Jupyter to measure the time taken to execute the cell. Sometimes you want to know how much it takes on a sample before extrapolating, in order to decide if your code needs optimization.

In [4]:
%%time

prices = prices_to_data_frame('data/PrixCarburants_T12019.xml')

CPU times: user 1min 4s, sys: 12.8 s, total: 1min 17s
Wall time: 1min 26s


In [5]:
prices.head()

,cp,id,latitude,longitude,maj,nom,pop,valeur
0,01000,1000001,4620114,519791,2019-01-04T10:53:48,Gazole,R,1328
1,01000,1000001,4620114,519791,2019-01-07T10:25:25,Gazole,R,1348
2,01000,1000001,4620114,519791,2019-01-10T08:54:28,Gazole,R,1374
3,01000,1000001,4620114,519791,2019-01-11T10:01:54,Gazole,R,1387
4,01000,1000001,4620114,519791,2019-01-14T11:04:53,Gazole,R,1394


In [6]:
%%time

prices.to_csv('output/prices.csv', sep=';', index=False)

CPU times: user 16.7 s, sys: 10.5 s, total: 27.2 s
Wall time: 1min 10s


In [7]:
!head output/prices.csv

cp;id;latitude;longitude;maj;nom;pop;valeur
01000;1000001;4620114;519791;2019-01-04T10:53:48;Gazole;R;1328
01000;1000001;4620114;519791;2019-01-07T10:25:25;Gazole;R;1348
01000;1000001;4620114;519791;2019-01-10T08:54:28;Gazole;R;1374
01000;1000001;4620114;519791;2019-01-11T10:01:54;Gazole;R;1387
01000;1000001;4620114;519791;2019-01-14T11:04:53;Gazole;R;1394
01000;1000001;4620114;519791;2019-01-16T10:21:21;Gazole;R;1394
01000;1000001;4620114;519791;2019-01-17T08:30:12;Gazole;R;1391
01000;1000001;4620114;519791;2019-01-18T10:08:25;Gazole;R;1401
01000;1000001;4620114;519791;2019-01-21T10:30:12;Gazole;R;1401
